# STAT 344 Group Project

### Estimates for SRS

In [1]:
library(tidyverse)
library(dplyr)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.3.6      v purrr   0.3.5 
v tibble  3.1.8      v dplyr   1.0.10
v tidyr   1.2.1      v stringr 1.4.1 
v readr   2.1.3      v forcats 0.5.2 
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
city<-read.csv("cities_srs.csv")
head(city)

,X,city,city_ascii,country,iso2,capital,population,continent,word_count,views,views_proportion
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<int>
1,3831,Coquitlam,Coquitlam,Canada,CA,,139284,NA,5418,272,0
2,9113,Lorengau,Lorengau,Papua New Guinea,PG,admin,5806,OC,409,16,0
3,702,Asmara,Asmara,Eritrea,ER,primary,963000,AF,2557,495,0
4,2285,St. Petersburg,St. Petersburg,United States,US,,265351,NA,11938,169,0
5,7610,Fazakerley,Fazakerley,United Kingdom,GB,,16786,EU,277,56,0
6,27739,Nemyriv,Nemyriv,Ukraine,UA,minor,11697,EU,575,10,0


In [3]:
len<-nrow(city)
FPC <- 1-(len/4205)
FPC

[1] 0.9762188

FPC is small enough to be ignored, but we will use it since we have calculated it

In [4]:
### SRS estimate +sd (with FPC))

summary<- tibble(city)%>%
        summarize(mean_word_count = mean(word_count),
                  sd_word_count = sqrt(var(word_count)*FPC))

summary

mean_word_count,sd_word_count
<dbl>,<dbl>
943.58,2003.305


In [5]:
#Just checking how many cities are there for each continent in our SRS
city%>%group_by(continent)%>%summarize(no_of_countries = n())

continent,no_of_countries
<chr>,<int>
AF,5
AS,27
EU,43
OC,2
SA,7
NA,16


> _*TEAM NOTE*_ : for the proportion calculation, i have tried calculating the avg number of words for cities from Asia. and checking *proportion* of cities with word count above this number

In [6]:
avg_wc_asia<-tibble(city)%>%filter(continent == "AS")%>%summarize(mean = mean(word_count))%>%as.numeric()
avg_wc_asia

[1] 609.0741

In [7]:
avg_wiki_article_len<- 644

In [8]:
city_prop<- city%>%mutate(bin_var = ifelse(word_count>avg_wiki_article_len, "yes", "no"))
head(city_prop)

,X,city,city_ascii,country,iso2,capital,population,continent,word_count,views,views_proportion,bin_var
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<int>,<chr>
1,3831,Coquitlam,Coquitlam,Canada,CA,,139284,NA,5418,272,0,yes
2,9113,Lorengau,Lorengau,Papua New Guinea,PG,admin,5806,OC,409,16,0,no
3,702,Asmara,Asmara,Eritrea,ER,primary,963000,AF,2557,495,0,yes
4,2285,St. Petersburg,St. Petersburg,United States,US,,265351,NA,11938,169,0,yes
5,7610,Fazakerley,Fazakerley,United Kingdom,GB,,16786,EU,277,56,0,no
6,27739,Nemyriv,Nemyriv,Ukraine,UA,minor,11697,EU,575,10,0,no


In [9]:
len<- nrow(city_prop)
p<- city_prop%>%
        filter(bin_var == "yes")%>%
        summarize(p_hat = n()/len,
                 sd = sqrt(FPC*(n()/len)*(1-n()/len)/len))
p

p_hat,sd
<dbl>,<dbl>
0.27,0.04386487


### Estimates for SRS

In [10]:
city_strata<-read.csv("cities_strat.csv")
head(city_strata)

,X,city,city_ascii,country,iso2,capital,population,continent,word_count,views,views_proportion
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<int>
1,13378,Euclid,Euclid,United States,US,,46550,NA,2133,272,0
2,14119,Linton Hall,Linton Hall,United States,US,,41386,NA,705,16,0
3,36241,Springs,Springs,United States,US,,7036,NA,780,495,0
4,18727,Waukee,Waukee,United States,US,,24089,NA,948,169,0
5,36726,Tabernacle,Tabernacle,United States,US,,6851,NA,2198,56,0
6,24751,Fostoria,Fostoria,United States,US,,14458,NA,26,10,0


In [11]:
city_strata%>%
        group_by(continent)%>%
        summarize(mean = mean(word_count),
                  sd = sd(word_count),
                  no_of_cities = n())%>%
### 1% of the populaiton is from OC continent, so only 1 city is from this continent in our DF
### this means the resulting sd of the wordcount = NA (devision by 0)
### this is thus replaced with the value 0
        replace_na(list(sd = 0))%>%
        mutate(weighted_mean = mean*(no_of_cities/len),
              weighted_var = (no_of_cities/len)^2*sd)%>%
        summarize(mu_hat_str = sum(weighted_mean),
                 sd_str = sqrt(FPC*sum(weighted_var)))

mu_hat_str,sd_str
<dbl>,<dbl>
970.61,22.59988


**NOTE** that our sd has become very small now and our mu_hat is more less the same

In [12]:
avg_wc_asia<- tibble(city_strata)%>%filter(continent == "AS")%>%summarize(mean = mean(word_count))%>%as.numeric()
avg_wc_asia

[1] 461.95

> _*TEAM NOTE*_: note that the avg word count for asian cities is very different now. This our proportion estimation will also be very different. Since rn we have 2 different random values (mean of word count of asian cities) which are used to calculate the proportions, our estimates for proportion will be biased + have extra variance due to this RV. We need another estimate to calculate the proportions, suggestions are welcome.

In [13]:
city_prop_str<- city_strata%>%mutate(bin_var = ifelse(word_count>avg_wiki_article_len, "yes", "no"))
head(city_prop)

,X,city,city_ascii,country,iso2,capital,population,continent,word_count,views,views_proportion,bin_var
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<int>,<chr>
1,3831,Coquitlam,Coquitlam,Canada,CA,,139284,NA,5418,272,0,yes
2,9113,Lorengau,Lorengau,Papua New Guinea,PG,admin,5806,OC,409,16,0,no
3,702,Asmara,Asmara,Eritrea,ER,primary,963000,AF,2557,495,0,yes
4,2285,St. Petersburg,St. Petersburg,United States,US,,265351,NA,11938,169,0,yes
5,7610,Fazakerley,Fazakerley,United Kingdom,GB,,16786,EU,277,56,0,no
6,27739,Nemyriv,Nemyriv,Ukraine,UA,minor,11697,EU,575,10,0,no


In [15]:
city_prop%>%
group_by(continent)%>%
summarize(p=sum(bin_var=="yes")/n(),
         var_str = p*(1-p)/n(),
         ratio = n()/len,
         p_r = p*ratio,
         var_r = var_str*ratio^2)%>%
        summarize(p_hat = sum(p_r),
                  se_str = sqrt(sum(var_r)*FPC))


p_hat,se_str
<dbl>,<dbl>
0.27,0.0419206
